In [1]:
import sys
sys.path.insert(0,'..')
import sib
import pandas as pd
import numpy as np

## Order of parameters

### Contacts:

in order: (i, j, t, lambda)

### Observations:
(i,s,t) which is in order:

- node
- state
- time

In [2]:
#contacts list

contacts = [ 
    (2,3,0,0.02),
    (2,3,0,0.02),
    (1,2,0,0.02),
    (2,1,1,0.02),
    (1,2,1,0.02),
    (0,1,3,0.02),
]
tot_contacts = len(contacts);
print(f"number of unique contacts: {tot_contacts}")

number of unique contacts: 6


In [22]:
#Partial observation on individuals

observations = [
       (1,1,1),
       (2,1,2),
]
tot_obs = len(observations)
print(f"number of observartions {tot_obs}")

number of observartions 2


In [23]:
# set dummy observation for computing marginals at time t_obs

nodes = np.unique(np.array(contacts)[:, 0:2])
t_obs = 3
observations += [(int(i), -1, t_obs) for i in nodes]

In [24]:
#setting the paramters of the inference model

params = sib.Params(prob_r=sib.Exponential(mu = 0.2), pseed=0.01)

In [25]:
f = sib.FactorGraph(contacts=contacts, observations=observations, params=params)

In [26]:
sib.iterate(f, maxit=100,tol=1e-5)

0 0.752266342467399
1 0.952266342467399
2 0.004872461483719004
3 0.0


## Marginal at time T = 2

In [28]:
for i in range(len(f.nodes)):
    n = f.nodes[f.get_index(i)]
    ttrue = list(n.times).index(t_obs)-1
    M = n.marginal_t(ttrue)
    print(f"Marginal of node {i}:")
    print(f"prob S: {M[0]:.3}")
    print(f"prob I: {M[1]:.3}")
    print(f"prob R: {M[2]:.3}\n")

Marginal of node 0:
prob S: 0.98
prob I: 0.00881
prob R: 0.0108

Marginal of node 1:
prob S: 0.0
prob I: 0.67
prob R: 0.33

Marginal of node 2:
prob S: 0.0
prob I: 0.819
prob R: 0.181

Marginal of node 3:
prob S: 0.975
prob I: 0.0117
prob R: 0.0132

